In [109]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def get_lin_coeff(col):
    coeffs = [(0, 0)]
    pts = zip(df.index, df[col])
    pt1 = next(pts)
    for pt in pts:
        m, b = linear_eq(pt1, pt)
        coeffs.append((m, b))
#         print(pt1[0], pt[0])
#         plt.plot(np.arange(pt1[0], pt[0]+1), m*np.arange(pt1[0], pt[0]+1) + b)
#         print(f'{m:.3}x + {b:.3}')
        pt1 = pt
    coeffs[0] = coeffs[1]
    return coeffs

def linear_eq(pt1, pt2):
    x1, y1 = pt1
    x2, y2 = pt2
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
    return np.round(m, 5), np.round(b, 5)

def get_zone(sb, pa):
    interval = np.where(df.index <= pa)[0][-1]
#     print(interval)
    
    l_coeff = df['Low Coeff'].iloc[interval]
    li_coeff = df['Low Intermediate Coeff'].iloc[interval]
    hi_coeff = df['High Intermediate Coeff'].iloc[interval]
    
#     print(l_coeff, li_coeff, hi_coeff)
    
    upper_l = l_coeff[0] * pa + l_coeff[1]
    upper_li = li_coeff[0] * pa + li_coeff[1]
    upper_hi = hi_coeff[0] * pa + hi_coeff[1]
    
    ok_l = sb < upper_l
    ok_li = sb < upper_li
    ok_hi = sb < upper_hi
#     print(upper_l, upper_li, upper_hi)
#     print(ok_l, ok_li, ok_hi)
    if ok_l and ok_li and ok_hi:
        msg = 'Low Risk Zone'
    elif ok_li and ok_hi:
        msg = 'Low Intermediate Risk Zone'
    elif ok_hi:
        msg = 'High Intermediate Risk Zone'
    else:
        msg = 'High Risk Zone'
    return 'Suggestion: ' + msg

def gitGud(b):
    out.clear_output()
    if not (0 <= sb.value):
        stat.value = 'ERROR: Serum Bilirubin (mg/dl) must be a non-negative real number.'
        return
    if not (12 <= pa.value <= 144):
        stat.value = 'ERROR: Postnatal Age (hours) must be between 12 and 144 hours, inclusive.'
        return
    stat.value = get_zone(sb.value, pa.value)
    
    plt.figure(figsize=(15,8))
    plt.scatter(pa.value, sb.value, s=80, color='red', label='You are here!')
    plt.plot(df.index, df['Low'], '--', label='Low')
    plt.plot(df.index, df['Low Intermediate'], '--', label='Low Intermediate')
    plt.plot(df.index, df['High Intermediate'], '--',label='High Intermediate')
    plt.ylabel('Serum Bilirubin (mg/dl)',fontsize=15)
    plt.xlabel('Postnatal Age (hours)',fontsize=15)
    plt.title('Neonatal Jaundice Graph',fontsize=20)
    plt.xticks(df.index)
    plt.legend()
    plt.grid()
    with out:
        plt.show()

if __name__ == "__main__":
    df = pd.read_excel('./Grading_jaundice.xlsx', index_col=0)
    df['Low Coeff'] = get_lin_coeff('Low')
    df['Low Intermediate Coeff'] = get_lin_coeff('Low Intermediate')
    df['High Intermediate Coeff'] = get_lin_coeff('High Intermediate')
    
    sb = widgets.FloatText()
    pa = widgets.FloatText(value=12)
    btn = widgets.Button(description='Do me a favor!')
    btn.on_click(gitGud)
    
    box_sb = widgets.HBox([widgets.Label(value='Serum Bilirubin (mg/dl): '), sb, widgets.Label(value='Postnatal Age (hours): '), pa])
    stat = widgets.Label()
    out = widgets.Output()
    
    display(box_sb)
    display(btn)
    display(stat)
    display(out)

Button(description='Do me a favor!', style=ButtonStyle())

Label(value='')

Output()